<a href="https://colab.research.google.com/github/vanesalinas/HenryPI_MLOps/blob/main/ETL%20y%20EDA/ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import gzip
import json
import ast
from pandas import json_normalize


# ***EXTRACTION (EXTRACCION)***

Extraemos los datos desde los archvos JSON proporcionados

In [2]:
# Lista para almacenar los datos del archivo JSON
fila = []

# Abrir el archivo comprimido con gzip y leer el contenido
with gzip.open('/content/steam_games.json.gz', 'rt') as games:
    for linea in games.readlines():
        # Cargar cada línea como un objeto JSON
        fila.append(json.loads(linea))

# Crear un DataFrame desde la lista de diccionarios
steam_games = pd.DataFrame(fila)

steam_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Nos encontramos con datos anidados dentro de los dataframes por lo que creamos una funcion para desanidar dichos datos.

In [3]:
def descomprimirjson(ruta, variable_anidada):
    '''Función que recibe una ruta de acceso a un archivo json anidado y carga la información en un
    DataFrame de Pandas'''
    fila = []
    with gzip.open(ruta, 'rt', encoding='MacRoman') as archivo:
        for line in archivo.readlines():
            fila.append(ast.literal_eval(line))

    df = pd.DataFrame(fila)
    df = df.explode(variable_anidada).reset_index()
    df = df.drop(columns="index")
    df = pd.concat([df, pd.json_normalize(df[variable_anidada])], axis=1)
    df = df.drop(columns=variable_anidada)

    return df

In [4]:
user_reviews = descomprimirjson("/content/user_reviews.json.gz",'reviews')
user_reviews.head() #mostrar las primeras filas del dataframe

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [5]:
user_items = descomprimirjson("/content/users_items.json.gz",'items')
user_items.head() #mostrar las primeras filas del dataframe

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7.0,0.0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0.0,0.0


# **TRANSFORMATION (TRANSFORMACION)**

Para este MVP no se piden transformaciones de datos pero se trabajara en leer el dataset con el formato correcto, el manejo de datos faltantes, eliminar duplicados y eliminar las columnas que no se necesitan para responder las consultas o preparar los modelos de aprendizaje automático, y de esa manera optimizar el rendimiento de la API y el entrenamiento del modelo.

In [6]:
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


In [7]:
user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59333 non-null  object
 1   user_url     59333 non-null  object
 2   funny        59305 non-null  object
 3   posted       59305 non-null  object
 4   last_edited  59305 non-null  object
 5   item_id      59305 non-null  object
 6   helpful      59305 non-null  object
 7   recommend    59305 non-null  object
 8   review       59305 non-null  object
dtypes: object(9)
memory usage: 4.1+ MB


In [8]:
user_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 8 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   user_url          object 
 4   item_id           object 
 5   item_name         object 
 6   playtime_forever  float64
 7   playtime_2weeks   float64
dtypes: float64(2), int64(1), object(5)
memory usage: 315.6+ MB


# **Manejo de valores faltantes**

Eliminamos filas que sean completamente nulas

In [9]:
clean_steam_games = steam_games.dropna(how='all')
clean_user_reviews = user_reviews.dropna(how='all')
clean_user_items = user_items.dropna(how='all')

Analizando los requerimientos para las funciones que se crearan luego, eliminamos los datos nulos o vacios de ciertas columnas que sin datos no nos proporcionan informacion, como por ejemplo developer.

In [10]:
clean_steam_games = clean_steam_games.dropna(subset=['developer','genres','id'])
clean_user_reviews = clean_user_reviews.dropna(subset=['user_id', 'item_id', 'review'])
clean_user_items = clean_user_items.dropna(subset=['user_id', 'item_id', 'playtime_forever'])

# **Eliminamos columnas que no utilizaremos**


In [11]:
clean_steam_games = clean_steam_games.drop(['publisher', 'title', 'url', 'tags', 'reviews_url', 'specs', 'early_access'], axis=1)
clean_user_reviews = clean_user_reviews.drop(['user_url', 'funny', 'last_edited', 'helpful'], axis=1)
clean_user_items = clean_user_items.drop(['items_count', 'steam_id', 'user_url', 'playtime_2weeks'], axis=1)

# **Valores duplicados**

Verificamos si existen valores duplicados para luego eliminarlos.

In [12]:
clean_steam_games['genres'] = clean_steam_games['genres'].apply(str)
clean_steam_games['genres'] = clean_steam_games['genres'].str.strip('[]')

In [13]:
#Contar si hay valores duplicados
duplicates_steam_games = clean_steam_games.duplicated().sum()
duplicates_user_reviews = clean_user_reviews.duplicated().sum()
duplicates_user_items = clean_user_items.duplicated().sum()

print(duplicates_steam_games, duplicates_user_reviews, duplicates_user_items)

1 874 59117


In [14]:
#Eliminar los valores duplicados. El parámetro inplace nos permite indicar si queremos que los valores duplicados se eliminen directamente en el DataFrame
clean_steam_games.drop_duplicates(inplace=True)
clean_user_reviews.drop_duplicates(inplace=True)
clean_user_items.drop_duplicates(inplace=True)

In [15]:
clean_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28680 entries, 88310 to 120443
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   genres        28680 non-null  object
 1   app_name      28680 non-null  object
 2   release_date  28662 non-null  object
 3   price         27465 non-null  object
 4   id            28680 non-null  object
 5   developer     28680 non-null  object
dtypes: object(6)
memory usage: 1.5+ MB


In [16]:
clean_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58431 entries, 0 to 59332
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    58431 non-null  object
 1   posted     58431 non-null  object
 2   item_id    58431 non-null  object
 3   recommend  58431 non-null  object
 4   review     58431 non-null  object
dtypes: object(5)
memory usage: 2.7+ MB


In [17]:
clean_user_items.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5094092 entries, 0 to 5170013
Data columns (total 4 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   item_id           object 
 2   item_name         object 
 3   playtime_forever  float64
dtypes: float64(1), object(3)
memory usage: 194.3+ MB


In [18]:
clean_user_reviews

,user_id,posted,item_id,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...
59328,76561198312638244,Posted July 10.,70,True,a must have classic from steam definitely wort...
59329,76561198312638244,Posted July 8.,362890,True,this game is a perfect remake of the original ...
59330,LydiaMorley,Posted July 3.,273110,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,Posted July 20.,730,True,:D


# ***Analisis de sentimiento***

Crearemos la columna 'sentiment_analysis' aplicando análisis de sentimiento con NLP con la siguiente escala:


*   0 : reseñas negativas
*   1 : reseñas neutrales o si la reseña escrita esta ausente
*   2 : reseñas positivas

Esta nueva columna reemplazara la de user_reviews.review

In [19]:
from textblob import TextBlob

# Función para realizar análisis de sentimiento y asignar la etiqueta según la escala
def sentiment_analysis(text):
    if pd.isnull(text):
        return 1  # Si la reseña está ausente, se considera neutral
    else:
        analysis = TextBlob(text)
        if analysis.sentiment.polarity < 0:
            return 0  # Reseña negativa
        elif analysis.sentiment.polarity == 0:
            return 1  # Reseña neutra
        else:
            return 2  # Reseña positiva

# Aplicar la función a la columna 'review' y crear la nueva columna 'sentiment_analysis'
clean_user_reviews['sentiment_analysis'] = clean_user_reviews['review'].apply(sentiment_analysis)

In [20]:
clean_user_reviews

,user_id,posted,item_id,recommend,review,sentiment_analysis
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,2
1,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,2
2,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,2
3,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...,2
4,js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...,0
...,...,...,...,...,...,...
59328,76561198312638244,Posted July 10.,70,True,a must have classic from steam definitely wort...,2
59329,76561198312638244,Posted July 8.,362890,True,this game is a perfect remake of the original ...,2
59330,LydiaMorley,Posted July 3.,273110,True,had so much fun plaing this and collecting res...,2
59331,LydiaMorley,Posted July 20.,730,True,:D,2


In [21]:
clean_user_reviews = clean_user_reviews.drop(['review'], axis=1)

In [22]:
clean_steam_games = clean_steam_games.rename(columns={'id': 'item_id'})

In [23]:
# Convierte la columna 'release_date' a tipo datetime
clean_steam_games['release_date'] = pd.to_datetime(clean_steam_games['release_date'], errors='coerce')

# Elimina los registros con NaT en la columna 'release_date'
clean_steam_games = clean_steam_games.dropna(subset=['release_date'])

<ipython-input-23-d94dace5d5ac>:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  clean_steam_games['release_date'] = pd.to_datetime(clean_steam_games['release_date'], errors='coerce')


In [24]:
clean_steam_games

,genres,app_name,release_date,price,item_id,developer
88310,"'Action', 'Casual', 'Indie', 'Simulation', 'St...",Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro
88311,"'Free to Play', 'Indie', 'RPG', 'Strategy'",Ironbound,2018-01-04,Free To Play,643980,Secret Level SRL
88312,"'Casual', 'Free to Play', 'Indie', 'Simulation...",Real Pool 3D - Poolians,2017-07-24,Free to Play,670290,Poolians.com
88313,"'Action', 'Adventure', 'Casual'",弹炸人2222,2017-12-07,0.99,767400,彼岸领域
88315,"'Action', 'Adventure', 'Simulation'",Battle Royale Trainer,2018-01-04,3.99,772540,Trickjump Games Ltd
...,...,...,...,...,...,...
120439,"'Action', 'Adventure', 'Casual', 'Indie'",Kebab it Up!,2018-01-04,1.99,745400,Bidoniera Games
120440,"'Casual', 'Indie', 'Simulation', 'Strategy'",Colony On Mars,2018-01-04,1.99,773640,"Nikita ""Ghost_RUS"""
120441,"'Casual', 'Indie', 'Strategy'",LOGistICAL: South Africa,2018-01-04,4.99,733530,Sacada
120442,"'Indie', 'Racing', 'Simulation'",Russian Roads,2018-01-04,1.99,610660,Laush Dmitriy Sergeevich


# ***LOAD (CARGA)***

Se almacenan los datos obtenidos en el formato de destino

In [25]:
clean_user_items.to_json('clean_user_items.json', orient='records', lines=True)

In [26]:
clean_user_reviews.to_json('clean_user_reviews.json', orient='records', lines=True)

In [27]:
clean_steam_games.to_json('clean_steam_games.json', orient='records', lines=True)

In [30]:
# Nombre del archivo comprimido
nombre_archivo_comprimido = "clean_user_items.json.gz"

# Comprimir el archivo JSON usando gzip
with open('clean_user_items.json', "rb") as archivo_json:
    with gzip.open(nombre_archivo_comprimido, "wb") as archivo_comprimido:
        archivo_comprimido.writelines(archivo_json)

print("Archivo JSON comprimido con éxito.")

Archivo JSON comprimido con éxito.


In [31]:
# Nombre del archivo comprimido
nombre_archivo_comprimido = "clean_user_reviews.json.gz"

# Comprimir el archivo JSON usando gzip
with open('clean_user_reviews.json', "rb") as archivo_json:
    with gzip.open(nombre_archivo_comprimido, "wb") as archivo_comprimido:
        archivo_comprimido.writelines(archivo_json)

print("Archivo JSON comprimido con éxito.")

Archivo JSON comprimido con éxito.


In [32]:
# Nombre del archivo comprimido
nombre_archivo_comprimido = "clean_steam_games.json.gz"

# Comprimir el archivo JSON usando gzip
with open('clean_steam_games.json', "rb") as archivo_json:
    with gzip.open(nombre_archivo_comprimido, "wb") as archivo_comprimido:
        archivo_comprimido.writelines(archivo_json)

print("Archivo JSON comprimido con éxito.")

Archivo JSON comprimido con éxito.
